In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse

In [ ]:
url_base='https://www.vghks.gov.tw/News.aspx?n=C03011BF96C680C4&sms=5EF61FB0D0F5B657'
url_work_table='RecruitInfo/3'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')